# Lab 01 · FastAPI, Vite, and Git

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A FastAPI backend is scaffolded with health and echo routes.
- A Vite React frontend is outlined with modern tooling defaults.
- A Git repository is initialized with environment templates.

## What will be learned
- The structure of a basic FastAPI service is reviewed.
- Development-time CORS settings are configured.
- Vite-powered React scaffolding steps are rehearsed.
- Git initialization practices are reinforced.


### FastAPI framework primer

**FastAPI in a nutshell.** [FastAPI](https://fastapi.tiangolo.com/) is an asynchronous Python web framework that automatically maps HTTP paths to Python callables called *path operations*. Typed request and response models built with [Pydantic](https://docs.pydantic.dev/) provide validation, while async support keeps expensive AI inference calls from blocking the event loop.

#### Core building blocks
1. **Application entry point** — Create a `FastAPI()` instance in `app/main.py` and register startup/shutdown handlers to open or close model clients. [Docs: First Steps](https://fastapi.tiangolo.com/tutorial/first-steps/)
2. **Routers and path operations** — Organize endpoints in `routers/` modules with `APIRouter` to keep chat, embeddings, and admin surfaces separate. Mount each router in `main.py` to expose `/api/chat`, `/api/embeddings`, etc. [Docs: APIRouter](https://fastapi.tiangolo.com/tutorial/bigger-applications/)
3. **Pydantic models** — Declare request/response schemas in `schemas.py` so prompts, system messages, and model options are validated before invoking inference. [Docs: Request Body](https://fastapi.tiangolo.com/tutorial/body/)
4. **Dependency injection** — Use `Depends` to pass shared services (OpenAI client, vector store, cache) into route functions without manually instantiating them. This keeps AI integrations testable and configurable. [Docs: Dependencies](https://fastapi.tiangolo.com/tutorial/dependencies/)
5. **Background tasks** — Offload long-running work such as audit logging, metrics collection, or follow-up summarization via `BackgroundTasks`. Students can immediately see how to chain AI workflows beyond the initial response. [Docs: Background Tasks](https://fastapi.tiangolo.com/tutorial/background-tasks/)
6. **Interactive OpenAPI docs** — FastAPI generates `/docs` and `/redoc` from your type hints, letting you verify model contracts, stream responses, and share live documentation with teammates. [Docs: Interactive API docs](https://fastapi.tiangolo.com/features/#interactive-api-docs)

#### Tooling and project layout for AI routes
- **Local server:** Run `uvicorn app.main:app --reload` for fast feedback while editing routers or schemas. Enable hot reload so AI inference tweaks are visible immediately. [Docs: Uvicorn](https://fastapi.tiangolo.com/deployment/uvicorn/)
- **Environment management:** Load `.env` files with `python-dotenv` or FastAPI settings management to inject API keys, base URLs, and model IDs without hardcoding secrets.
- **Folder structure:** Start with `app/main.py`, `app/routers/`, `app/schemas.py`, and `app/services/ai.py`. Keep retry utilities, tracing helpers, and vector database adapters in `services/` so routes remain thin controllers.
- **Testing hooks:** Leverage FastAPI's `TestClient` to write unit tests that mock AI providers and confirm each router handles success, errors, and streaming payloads.

### Vite + React framework primer

#### Why Vite for React AI UIs
The frontend labs rely on [Vite](https://vitejs.dev/guide/) to scaffold a React UI with lightning-fast hot module replacement, native ES module bundling, and first-class TypeScript. CLI commands such as `npm create vite@latest`, `npm install`, `npm run dev`, and `npm run build` (or their `pnpm`/`yarn` equivalents) keep iteration tight while preparing optimized builds for deployment.

#### Core tooling and project conventions
- **Dev server workflow:** `npm run dev` launches a Vite server that can [proxy](https://vitejs.dev/config/server-options.html#server-proxy) `/api` calls to FastAPI, eliminating CORS friction while you develop AI features.
- **Environment variables:** Prefix keys with `VITE_` (e.g., `VITE_API_BASE_URL`, `VITE_OPENAI_MODEL`) so they are exposed via `import.meta.env`. Use separate `.env.local` files for experimental model toggles.
- **Module organization:** Mirror backend routers with feature folders like `src/features/chat/`, `src/features/embeddings/`, and keep shared helpers under `src/lib/` (`apiClient.js`, `retry.js`, `streamParser.js`).
- **React hooks and state:** Follow the [React documentation](https://react.dev/learn) to combine `useState`, `useEffect`, and custom hooks that orchestrate fetch lifecycles. Patterns such as `useReducer` for complex prompt builders or `useRef` for streaming buffers keep AI UIs maintainable.
- **UI composition:** Component libraries (Mantine, Chakra) or design systems can slot into `src/components/` for reusable chat bubbles, token counters, and loading spinners.

#### Mini walkthrough: bootstrapping the AI stack
1. **Scaffold FastAPI** with `python -m venv .venv && pip install fastapi uvicorn` and create `app/main.py` that instantiates the app, registers routers, and mounts static files if needed.
2. **Add routers and schemas** under `app/routers/inference.py` with `Pydantic` models describing your prompt payloads and inference responses.
3. **Wire dependencies** like model clients via `Depends`, storing credentials in `.env` and loading them with a settings class.
4. **Run `uvicorn app.main:app --reload`** to expose `/docs`, test path operations, and confirm background tasks fire during inference.
5. **Create the Vite React frontend** with `npm create vite@latest frontend -- --template react` and install UI dependencies that render form controls and streaming areas.
6. **Configure Vite proxy and envs** in `vite.config.js` and `.env.local` so `fetch` helpers in `src/lib/apiClient.js` point to FastAPI during development.
7. **Implement React hooks and components** (e.g., `useInference` in `src/features/chat/useInference.js`) that call the backend, manage retries, and render streaming updates.
8. **Iterate locally** by running `uvicorn` and `npm run dev` together, validating requests through the autogenerated FastAPI docs and the Vite dev tools before containerization or deployment.



## Prerequisites & install
The container workflow depends on these locally installed tools:

```bash
docker --version
docker compose version
git --version
```

After cloning the repository, build and start the services with Docker Compose:

```bash
cd ai-web
docker compose build
docker compose up
# Backend → http://localhost:8000
# Frontend → http://localhost:5173

# Shut the stack down when finished exploring:
docker compose down
```

## Step-by-step tasks
Each step configures local source files that the Docker stack will mount so secrets remain outside the built images.

### Step 1: Backend folder layout and Dockerfile
A backend folder is created with starter FastAPI files and a Dockerfile that installs dependencies inside the image.

In [ ]:
from pathlib import Path
base = Path("ai-web/backend")
base.mkdir(parents=True, exist_ok=True)

# Ensure the application package exists so FastAPI can locate modules.
(base / "app").mkdir(parents=True, exist_ok=True)
(base / "app" / "__init__.py").write_text("")

# Provide a sample environment file that documents required secrets.
(base / ".env.example").write_text('# Environment variables (never commit real keys)GEMINI_API_KEY=')

# List Python dependencies that will be installed within the backend container.
(base / "requirements.txt").write_text('''fastapi
uvicorn[standard]
pydantic
python-dotenv
google-genai
faiss-cpu
numpy
''')

# Create the FastAPI entrypoint with health and echo routes.
(base / "app" / "main.py").write_text('''from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_methods=["*"],
    allow_headers=["*"],
)


class EchoIn(BaseModel):
    msg: str


@app.get("/health")
def health():
    return {"status": "ok"}


@app.post("/echo")
def echo(payload: EchoIn):
    return {"msg": payload.msg}
''')

# Define a Dockerfile that installs dependencies and starts uvicorn.
(base / "Dockerfile").write_text('''FROM python:3.11-slim

WORKDIR /app

ENV PYTHONDONTWRITEBYTECODE=1 \n    PYTHONUNBUFFERED=1

COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt

COPY app ./app

CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
''')

print("Backend scaffold and Dockerfile were written under ai-web/backend.")

### Step 2: Frontend placeholders and Dockerfile
Frontend placeholders are positioned so Vite React components (`src/App.jsx`, `src/main.jsx`) can be customized while the generated `vite.config.js` continues to manage dev server defaults. A lightweight Dockerfile installs Node.js dependencies for the Vite dev server.

In [ ]:
from pathlib import Path
frontend = Path("ai-web/frontend")
src = frontend / "src"
frontend.mkdir(parents=True, exist_ok=True)
src.mkdir(parents=True, exist_ok=True)

# Create a lib directory for shared utilities between components.
(src / "lib").mkdir(parents=True, exist_ok=True)

# Bootstrap a simple API helper with descriptive error handling.
(src / "lib" / "api.js").write_text('''const BASE = import.meta.env.VITE_API_BASE || "http://localhost:8000";

export async function post(path, body) {
  const res = await fetch(`${BASE}${path}`, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify(body)
  });
  if (!res.ok) {
    throw new Error(`HTTP ${res.status}`);
  }
  return res.json();
}
''')

# Author a basic demo interface that interacts with the echo endpoint.
(src / "App.jsx").write_text('''import { useState } from 'react';
import { post } from './lib/api';

function App() {
  const [msg, setMsg] = useState('hello');
  const [response, setResponse] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  async function handleSend() {
    setLoading(true);
    setError('');
    try {
      const json = await post('/echo', { msg });
      setResponse(json.msg);
    } catch (err) {
      setError(String(err));
    } finally {
      setLoading(false);
    }
  }

  return (
    <main style={{ padding: 24 }}>
      <h1>Lab 1 — Echo demo</h1>
      <input value={msg} onChange={(event) => setMsg(event.target.value)} />
      <button onClick={handleSend} disabled={loading}>Send</button>
      {loading && <p>Loading…</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}
      <pre>{response}</pre>
    </main>
  );
}

export default App;
''')

# Mount the App component using Vite's modern entry point.
(src / "main.jsx").write_text('''import React from 'react';
import ReactDOM from 'react-dom/client';
import App from './App.jsx';

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>,
);
''')

# Ensure Vite configuration and HTML entrypoint exist for the dev server.
(frontend / "vite.config.js").write_text('''import { defineConfig } from "vite";
import react from "@vitejs/plugin-react";

export default defineConfig({
  plugins: [react()],
  server: {
    host: true,
    port: 5173,
  },
});
''')

(frontend / "index.html").write_text('''<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Lab 1 Echo Demo</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.jsx"></script>
  </body>
</html>
''')

# Provide a package.json that mirrors the Vite React starter dependencies.
(frontend / "package.json").write_text('''{
  "name": "frontend",
  "version": "0.0.0",
  "private": true,
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "preview": "vite preview --host"
  },
  "dependencies": {
    "react": "^18.3.1",
    "react-dom": "^18.3.1"
  },
  "devDependencies": {
    "@vitejs/plugin-react": "^4.3.1",
    "vite": "^5.2.0"
  }
}
''')

# Provide a Dockerfile that installs dependencies and runs the Vite dev server.
(frontend / "Dockerfile").write_text('''FROM node:20-alpine

WORKDIR /app

COPY package*.json ./
RUN npm install

COPY . .

CMD ["npm", "run", "dev", "--", "--host"]
''')

print("Frontend placeholders and Dockerfile were written under ai-web/frontend.")

### Step 3: Docker Compose orchestration
A top-level `docker-compose.yml` is created so the backend and frontend can be started together with a single command.

In [ ]:
from pathlib import Path
compose = Path("ai-web/docker-compose.yml")
compose.parent.mkdir(parents=True, exist_ok=True)

# Define services that mount local source for rapid iteration while keeping dependencies inside the containers.
compose.write_text('''version: "3.9"

services:
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    environment:
      - GEMINI_API_KEY=${GEMINI_API_KEY:-}
    volumes:
      - ./backend/app:/app/app
      - ./backend/.env.example:/app/.env:ro

  frontend:
    build: ./frontend
    ports:
      - "5173:5173"
    environment:
      - VITE_API_BASE=http://localhost:8000
    volumes:
      - ./frontend/src:/app/src
      - ./frontend/vite.config.js:/app/vite.config.js
      - ./frontend/index.html:/app/index.html
      - ./frontend/package.json:/app/package.json
''')

print("Docker Compose file was written under ai-web/docker-compose.yml.")

### Step 4: Git initialization
Git is initialized locally so changes can be tracked.

### FastAPI ↔ Vite AI workflow checklist

1. Scaffold FastAPI with a dedicated router for AI inference (e.g., `routers/inference.py`) and register it in `main.py`.
2. Define `Pydantic` models for request payloads (prompt text, model options) and responses (structured outputs, tokens, metadata).
3. Configure dependency-injected clients for external AI providers, loading keys from `.env` files via `python-dotenv` or container secrets.
4. Enable CORS middleware and document the route via FastAPI's autogenerated docs to confirm payload contracts.
5. In Vite, create API utility modules that read backend URLs from `import.meta.env` and call the FastAPI endpoints.
6. Build React components that manage prompt forms, loading indicators, streaming updates, and error toasts tied to those utilities.
7. Use Vite's dev server proxy to forward `/api` requests to FastAPI for local iteration, then validate the flow through Docker Compose.
8. Update production Dockerfiles or CI pipelines so backend and frontend environments receive the same `.env` configuration for AI model credentials.


```bash
cd ai-web
git init
git add .
git commit -m "Lab 1 scaffold"
```

## Validation / acceptance checks
```bash
# locally
docker compose up -d
curl http://localhost:8000/health
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"hello"}'
docker compose down
```
- HTTP 200 responses include status "ok" and the echoed payload while the stack runs in Docker.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A README entry is expanded to document backend and frontend start commands.
- A GitHub repository is connected for remote backups.